In [1]:
!python -V

Python 3.9.12


In [19]:
import pandas as pd

In [26]:
import pickle

In [27]:
import seaborn as sns
import matplotlib.pyplot as plt

In [28]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [29]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

## Download the data

In [14]:
import os
# https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet
trip_url_base='https://s3.amazonaws.com/nyc-tlc/trip+data'
tripdata_filenames = ["green_tripdata_2021-01.parquet", "green_tripdata_2021-02.parquet"]
tripdata_filename_jan = tripdata_filenames[0]
tripdata_filename_feb = tripdata_filenames[1]

In [15]:
def download_datafile(url_base, filename):
    if not os.path.exists(f'data/{filename}'):
        print(f'Downloading {filename}')
        full_url = f'{url_base}/{filename}'
        !wget -P data $full_url
    else:
        print(f'File {filename} already exists!')
        
def download_alldata(url_base, filenamelist):
    for trip_filename in tripdata_filenames:
        download_datafile(trip_url_base, trip_filename)

In [17]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [16]:
!mkdir data

download_alldata(trip_url_base, tripdata_filenames)

mkdir: cannot create directory ‘data’: File exists
--2022-05-25 17:30:22--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.40.160
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.40.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1333519 (1.3M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_2021-01.parquet’

green_tripdata_2021 100%[===================>]   1.27M  1.10MB/s    in 1.2s    

2022-05-25 17:30:24 (1.10 MB/s) - ‘data/green_tripdata_2021-01.parquet’ saved [1333519/1333519]

--2022-05-25 17:30:24--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-02.parquet
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.165.24
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.165.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1145679 (1.1M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_2021-02.parquet

In [20]:
# pd.read_parquet(f'data/{tripdata_filename_jan}')
df_train = read_dataframe(f'data/{tripdata_filename_jan}')
df_val = read_dataframe(f'data/{tripdata_filename_feb}')

In [21]:
len(df_train), len(df_val)

(73908, 61921)

In [22]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [30]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [31]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [32]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715208269472

In [34]:
os.makedirs("models")
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [36]:
with mlflow.start_run():

    mlflow.set_tag("developer", "michal")

    mlflow.log_param("train-data-path", f'data/{tripdata_filename_jan}')
    mlflow.log_param("valid-data-path", f'data/{tripdata_filename_feb}')

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [37]:
import xgboost as xgb

In [38]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [39]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [40]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [41]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[19:50:39] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.                                                                                                                                        
[0]	validation-rmse:16.99035                                                                                                                                                                                                                                  
[1]	validation-rmse:13.91063                                                                                                                                                                                                                                  
[2]	validation-rmse:11.67377                                                                                                                                                                                                               

[31]	validation-rmse:6.53417                                                                                                                                                                                                                                  
[32]	validation-rmse:6.53255                                                                                                                                                                                                                                  
[33]	validation-rmse:6.53130                                                                                                                                                                                                                                  
[34]	validation-rmse:6.52951                                                                                                                                                                                                               

[63]	validation-rmse:6.49592                                                                                                                                                                                                                                  
[64]	validation-rmse:6.49474                                                                                                                                                                                                                                  
[65]	validation-rmse:6.49366                                                                                                                                                                                                                                  
[66]	validation-rmse:6.49321                                                                                                                                                                                                               

[95]	validation-rmse:6.46857                                                                                                                                                                                                                                  
[96]	validation-rmse:6.46787                                                                                                                                                                                                                                  
[97]	validation-rmse:6.46724                                                                                                                                                                                                                                  
[98]	validation-rmse:6.46633                                                                                                                                                                                                               

[127]	validation-rmse:6.45148                                                                                                                                                                                                                                 
[128]	validation-rmse:6.45156                                                                                                                                                                                                                                 
[129]	validation-rmse:6.45131                                                                                                                                                                                                                                 
[130]	validation-rmse:6.45112                                                                                                                                                                                                              

...

[536]	validation-rmse:6.37303                                                                                                                                                                                                                                 
[537]	validation-rmse:6.37279                                                                                                                                                                                                                                 
[538]	validation-rmse:6.37246                                                                                                                                                                                                                                 
[539]	validation-rmse:6.37218                                                                                                                                                                                                              

[568]	validation-rmse:6.36772                                                                                                                                                                                                                                 
[569]	validation-rmse:6.36761                                                                                                                                                                                                                                 
[570]	validation-rmse:6.36748                                                                                                                                                                                                                                 
[571]	validation-rmse:6.36726                                                                                                                                                                                                              

[600]	validation-rmse:6.36249                                                                                                                                                                                                                                 
[601]	validation-rmse:6.36217                                                                                                                                                                                                                                 
[602]	validation-rmse:6.36200                                                                                                                                                                                                                                 
[603]	validation-rmse:6.36184                                                                                                                                                                                                              

[632]	validation-rmse:6.35717                                                                                                                                                                                                                                 
[633]	validation-rmse:6.35693                                                                                                                                                                                                                                 
[634]	validation-rmse:6.35689                                                                                                                                                                                                                                 
[635]	validation-rmse:6.35645                                                                                                                                                                                                              

KeyboardInterrupt: 

In [42]:
mlflow.xgboost.autolog(disable=True)

In [43]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[19:55:10] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95634
[2]	validation-rmse:16.59114
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29011
[5]	validation-rmse:13.32800
[6]	validation-rmse:12.47570
[7]	validation-rmse:11.72140
[8]	validation-rmse:11.05888
[9]	validation-rmse:10.47583
[10]	validation-rmse:9.96483
[11]	validation-rmse:9.51784
[12]	validation-rmse:9.12620
[13]	validation-rmse:8.78522
[14]	validation-rmse:8.49037
[15]	validation-rmse:8.23332
[16]	validation-rmse:8.01179
[17]	validation-rmse:7.81822
[18]	validation-rmse:7.65209
[19]	validation-rmse:7.50743
[20]	validation-rmse:7.38235
[21]	validation-rmse:7.27382
[22]	validation-rmse:7.17865
[23]	validation-rmse:7.09606
[24]	validation-rmse:7.02481
[25]	validation-rmse:6.96336
[26]	validation-rmse:6.91001
[27]	validation-rmse:6.86273
[28]	validation-rmse:6.82111
[29]	validation-rmse:6.78450
[30]	valida

[273]	validation-rmse:6.38673
[274]	validation-rmse:6.38660
[275]	validation-rmse:6.38649
[276]	validation-rmse:6.38615
[277]	validation-rmse:6.38595
[278]	validation-rmse:6.38585
[279]	validation-rmse:6.38558
[280]	validation-rmse:6.38529
[281]	validation-rmse:6.38510
[282]	validation-rmse:6.38480
[283]	validation-rmse:6.38455
[284]	validation-rmse:6.38429
[285]	validation-rmse:6.38410
[286]	validation-rmse:6.38377
[287]	validation-rmse:6.38341
[288]	validation-rmse:6.38321
[289]	validation-rmse:6.38288
[290]	validation-rmse:6.38241
[291]	validation-rmse:6.38218
[292]	validation-rmse:6.38187
[293]	validation-rmse:6.38155
[294]	validation-rmse:6.38099
[295]	validation-rmse:6.38073
[296]	validation-rmse:6.38059
[297]	validation-rmse:6.38034
[298]	validation-rmse:6.38019
[299]	validation-rmse:6.38001
[300]	validation-rmse:6.37986
[301]	validation-rmse:6.37969
[302]	validation-rmse:6.37924
[303]	validation-rmse:6.37890
[304]	validation-rmse:6.37895
[305]	validation-rmse:6.37869
[306]	vali

[547]	validation-rmse:6.33680
[548]	validation-rmse:6.33659
[549]	validation-rmse:6.33672
[550]	validation-rmse:6.33653
[551]	validation-rmse:6.33657
[552]	validation-rmse:6.33651
[553]	validation-rmse:6.33631
[554]	validation-rmse:6.33623
[555]	validation-rmse:6.33601
[556]	validation-rmse:6.33609
[557]	validation-rmse:6.33599
[558]	validation-rmse:6.33582
[559]	validation-rmse:6.33582
[560]	validation-rmse:6.33561
[561]	validation-rmse:6.33555
[562]	validation-rmse:6.33554
[563]	validation-rmse:6.33540
[564]	validation-rmse:6.33525
[565]	validation-rmse:6.33519
[566]	validation-rmse:6.33517
[567]	validation-rmse:6.33522
[568]	validation-rmse:6.33518
[569]	validation-rmse:6.33510
[570]	validation-rmse:6.33491
[571]	validation-rmse:6.33472
[572]	validation-rmse:6.33458
[573]	validation-rmse:6.33443
[574]	validation-rmse:6.33427
[575]	validation-rmse:6.33401
[576]	validation-rmse:6.33395
[577]	validation-rmse:6.33392
[578]	validation-rmse:6.33378
[579]	validation-rmse:6.33362
[580]	vali

[821]	validation-rmse:6.31328
[822]	validation-rmse:6.31317
[823]	validation-rmse:6.31322
[824]	validation-rmse:6.31317
[825]	validation-rmse:6.31318
[826]	validation-rmse:6.31317
[827]	validation-rmse:6.31317
[828]	validation-rmse:6.31309
[829]	validation-rmse:6.31290
[830]	validation-rmse:6.31278
[831]	validation-rmse:6.31271
[832]	validation-rmse:6.31266
[833]	validation-rmse:6.31270
[834]	validation-rmse:6.31273
[835]	validation-rmse:6.31271
[836]	validation-rmse:6.31268
[837]	validation-rmse:6.31260
[838]	validation-rmse:6.31253
[839]	validation-rmse:6.31239
[840]	validation-rmse:6.31230
[841]	validation-rmse:6.31227
[842]	validation-rmse:6.31215
[843]	validation-rmse:6.31216
[844]	validation-rmse:6.31220
[845]	validation-rmse:6.31214
[846]	validation-rmse:6.31200
[847]	validation-rmse:6.31178
[848]	validation-rmse:6.31167
[849]	validation-rmse:6.31149
[850]	validation-rmse:6.31131
[851]	validation-rmse:6.31137
[852]	validation-rmse:6.31137
[853]	validation-rmse:6.31140
[854]	vali

/home/michal/.virtualenvs/mlflow/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [44]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", f'data/{tripdata_filename_jan}')
        mlflow.log_param("valid-data-path", f'data/{tripdata_filename_feb}')
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2022/05/25 19:57:46 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
/home/michal/.virtualenvs/mlflow/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
